In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Input, MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Only keep samples for airplane(0), automobile(1), ship(8), truck(9)
selected_classes = [0, 1, 8, 9]
class_map = {0: 0, 1: 1, 8: 2, 9: 3}  # Map original labels to 0-3

def filter_classes(x, y):
    mask = np.isin(y, selected_classes).flatten()
    x_filtered = x[mask]
    y_filtered = y[mask]
    # Remap labels to 0-3
    y_filtered = np.vectorize(class_map.get)(y_filtered)
    return x_filtered, y_filtered

x_train, y_train = filter_classes(x_train, y_train)
x_test, y_test = filter_classes(x_test, y_test)

# Normalize data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding for 4 classes
y_train_onehot = to_categorical(y_train, 4)
y_test_onehot = to_categorical(y_test, 4)

# Define the same architecture but with 4 outputs
INPUT_SHAPE = (32, 32, 3)
KERNEL_SIZE = (3, 3)
L2_REG = 1e-4

model = Sequential()


# Edge Detection Block - Important for ship contours
model.add(Conv2D(32, (5, 5), activation='relu', padding='same', input_shape=INPUT_SHAPE, kernel_regularizer=l2(L2_REG)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Texture Analysis Block - For water/wave patterns
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Shape Detection Block - For ship silhouettes
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Context Integration Block - For understanding ship environment
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Classification Layers
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(L2_REG)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(L2_REG)))
model.add(Dropout(0.2))
# Change to 4 outputs with softmax activation for multiclass
model.add(Dense(4, activation='softmax', kernel_regularizer=l2(L2_REG)))

# Compile with categorical_crossentropy loss
model.compile(
    loss=CategoricalCrossentropy(),
    optimizer='adam',
    metrics=['accuracy']
)

# INPUT_SHAPE = (32, 32, 3)
# KERNEL_SIZE = (3, 3)
# L2_REG = 1e-4

# model = Sequential()
# model.add(Conv2D(64, (5, 5), activation='relu', padding='same', input_shape=INPUT_SHAPE, kernel_regularizer=l2(L2_REG)))
# model.add(BatchNormalization())
# model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
# model.add(BatchNormalization())
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
# model.add(BatchNormalization())
# model.add(Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))

# model.add(Conv2D(8, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
# model.add(BatchNormalization())
# model.add(Conv2D(8, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(L2_REG)))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))

# model.add(GlobalAveragePooling2D())
# model.add(Dense(16, activation='relu', kernel_regularizer=l2(L2_REG)))
# model.add(Dropout(0.3))
# model.add(Dense(8, activation='relu', kernel_regularizer=l2(L2_REG)))
# model.add(Dropout(0.2))
#model.add(Dense(4, activation='softmax', kernel_regularizer=l2(L2_REG)))


C:\Users\Laptop\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training multiclass classifier on CIFAR-10 (airplane, automobile, ship, truck)...
Epoch 1/20


C:\Users\Laptop\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 61s 21ms/step - accuracy: 0.3068 - loss: 1.3782 - val_accuracy: 0.4565 - val_loss: 1.1172 - learning_rate: 0.0010
Epoch 2/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 58s 23ms/step - accuracy: 0.3963 - loss: 1.2306 - val_accuracy: 0.4548 - val_loss: 1.0831 - learning_rate: 0.0010
Epoch 3/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 99s 40ms/step - accuracy: 0.4074 - loss: 1.1728 - val_accuracy: 0.4512 - val_loss: 1.0476 - learning_rate: 0.0010
Epoch 4/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 79s 31ms/step - accuracy: 0.4231 - loss: 1.1424 - val_accuracy: 0.4428 - val_loss: 1.0547 - learning_rate: 0.0010
Epoch 5/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 52s 21ms/step - accuracy: 0.4210 - loss: 1.1281 - val_accuracy: 0.4320 - val_loss: 1.0582 - learning_rate: 0.0010
Epoch 6/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 52s 21ms/step - accuracy: 0.4312 - loss: 1.0998 - val_accuracy: 0.4545 - val_loss: 0.9905 - learning_rate: 0.0010
Epoch 7/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 57s 23ms/step - accuracy: 0.4343 

KeyboardInterrupt: 

In [ ]:
# Apply Cutout augmentation
def apply_cutout(image, size=8, n_holes=1):
    h, w = image.shape[0], image.shape[1]
    for n in range(n_holes):
        # Random position of cutout
        y = np.random.randint(h)
        x = np.random.randint(w)
        
        # Ensure cutout stays within image bounds
        y1 = np.clip(y - size // 2, 0, h)
        y2 = np.clip(y + size // 2, 0, h)
        x1 = np.clip(x - size // 2, 0, w)
        x2 = np.clip(x + size // 2, 0, w)
        
        # Set the cutout region to zero
        image[y1:y2, x1:x2, :] = 0
    return image

# Apply cutout to training data
x_train_cutout = x_train.copy()
for i in range(len(x_train_cutout)):
    x_train_cutout[i] = apply_cutout(x_train_cutout[i])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=8,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)
datagen.fit(x_train_cutout)


callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

# Train the model
print("Training multiclass classifier on CIFAR-10 (airplane, automobile, ship, truck)...")
history = model.fit(
    datagen.flow(x_train_cutout, y_train_onehot, batch_size=8),
    epochs=20,
    validation_data=(x_test, y_test_onehot),
    callbacks=callbacks,
    steps_per_epoch=len(x_train_cutout) // 8,
    verbose=1
)

# Save model
model.save('cifar10_multiclass_model_4class.keras')

# Evaluate the model
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.squeeze(y_test)

# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
class_names = ['airplane', 'automobile', 'ship', 'truck']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
plt.figure(figsize=(8, 8))
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for CIFAR-10 (4-class) Classification')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.tight_layout()
plt.show()


In [6]:
import os

# Calculate model size (in bytes and MB)
model.save('temp_model_for_size.keras', include_optimizer=False)
model_size_bytes = os.path.getsize('temp_model_for_size.keras')
model_size_mb = model_size_bytes / (1024 * 1024)
print(f"Model size on disk: {model_size_bytes:,} bytes ({model_size_mb:.2f} MB)")

# Calculate network size (number of parameters)
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
non_trainable_params = np.sum([np.prod(v.shape) for v in model.non_trainable_weights])
total_params = trainable_params + non_trainable_params
print(f"Trainable parameters: {trainable_params:,}")
print(f"Non-trainable parameters: {non_trainable_params:,}")
print(f"Total parameters (network size): {total_params:,}")



Model size on disk: 3,035,661 bytes (2.90 MB)
Trainable parameters: 241,060
Non-trainable parameters: 896
Total parameters (network size): 241,956


KeyError: "Registering two statistical functions with name 'AddV2,flops'! (Previous registration was in register C:\\Users\\Laptop\\AppData\\Roaming\\Python\\Python312\\site-packages\\tensorflow\\python\\framework\\registry.py:65)"

In [ ]:
import tensorflow as tf

concrete_func = tf.function(lambda x: model(x)).get_concrete_function(
    tf.TensorSpec([1, 32, 32, 3], tf.float32))

# Get FLOPs via TF Profiler (experimental)
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

profile_opts = ProfileOptionBuilder.float_operation()
flops = profile(concrete_func.graph, options=profile_opts)
print(f"FLOPs: {flops.total_float_ops:,}")


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
FLOPs: 64,330,904
